In [ ]:
import pandas as pd

# Paste the path inside the quotes
# It usually looks like '/content/your_file.csv'
df = pd.read_csv(r"E:\College\Term 5\AI\Dataset\AI_Project\data.csv")

print(df.columns)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Check for missing values
print("Missing Values:\n", df.isnull().sum())

# 2. Summary Statistics (Average salary, max risk, etc.)
print("\nSummary Statistics:")
display(df.describe())

# 3. Visualize the Automation Risk
plt.figure(figsize=(8, 5))
sns.histplot(df['Automation Risk (%)'], bins=20, kde=True)
plt.title('Distribution of Automation Risk')
plt.xlabel('Automation Risk (%)')
plt.ylabel('Count')
plt.show()

In [ ]:
# Create a new column 'Human_Need'
# 1 = Safe (Human is needed)
# 0 = At Risk (Machine will replace)

def classify_risk(risk):
    if risk < 50:
        return 1  # High Human Need
    else:
        return 0  # Low Human Need

# Apply this rule to the data
df['Human_Need'] = df['Automation Risk (%)'].apply(classify_risk)

print("Target Column 'Human_Need' created successfully!")

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create the encoder
le = LabelEncoder()

# List of columns that are currently text
text_cols = ['Job Title', 'Industry', 'Required Education', 'Location', 'Job Status', 'AI Impact Level']

# Convert them
for col in text_cols:
    df[col] = le.fit_transform(df[col])

print("All text converted to numbers. Here is the final table:")
display(df.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Define Features (X) and Target (y)
# We DROP 'Automation Risk (%)' because that gives away the answer!
# We DROP 'Human_Need' because that is what we are predicting.
X = df.drop(columns=['Automation Risk (%)', 'Human_Need'])
y = df['Human_Need']

# 2. Split the data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initialize the 3 Models
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier()
}

# 4. Train and Evaluate
print("Model Results:\n" + "-"*30)
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Calculate accuracy
    acc = accuracy_score(y_test, predictions)
    print(f"{name} Accuracy: {acc*100:.2f}%")

In [ ]:
!pip install streamlit -q

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# ==========================================
# PASTE YOUR PATH BELOW (Keep the r before the quotes!)
# Example: file_path = r"E:\College\Term 5\AI\Dataset\AI_Project\data.csv"
# ==========================================
file_path = r"E:\College\Term 5\AI\Dataset\AI_Project\data.csv"
# ^^^ DELETE "PASTE_YOUR_PATH_HERE" AND PASTE (Ctrl+V) 


# --- APP SETUP ---
st.title("🤖 Job Automation Predictor")

@st.cache_data
def get_model():
    try:
        # Load the file using the specific path
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        return None
    except OSError:
        # This handles if the path includes quotes inside the string
        try:
            clean_path = file_path.replace('"', '')
            df = pd.read_csv(clean_path)
        except:
            return None

    # Preprocessing
    le = LabelEncoder()
    cols = ['Job Title', 'Industry', 'Required Education', 'Location', 'Job Status', 'AI Impact Level']
    for col in cols:
        if col in df.columns: df[col] = le.fit_transform(df[col])
            
    df['Human_Need'] = df['Automation Risk (%)'].apply(lambda x: 1 if x < 50 else 0)
    X = df[['Median Salary (USD)', 'Remote Work Ratio (%)', 'AI Impact Level', 'Experience Required (Years)']]
    y = df['Human_Need']
    model = RandomForestClassifier()
    model.fit(X, y)
    return model

model = get_model()

# --- INPUTS & PREDICTION ---
st.sidebar.header("Job Features")
salary = st.sidebar.slider("Median Salary ($)", 30000, 150000, 80000)
remote = st.sidebar.slider("Remote Work Ratio (%)", 0, 100, 50)
ai = st.sidebar.selectbox("AI Impact Level (1=Low, 5=High)", [1, 2, 3, 4, 5])
exp = st.sidebar.slider("Years of Experience", 0, 20, 5)

if model is None:
    st.error(f"🚨 ERROR: Cannot find the file at this path:\n\n{file_path}\n\nPlease check the path in your code.")
elif st.button("Predict"):
    prediction = model.predict([[salary, remote, ai, exp]])
    if prediction[0] == 1: st.success("✅ SAFE: High Human Need")
    else: st.error("⚠️ RISK: Likely Automated")